In [ ]:
# import torch
# import transformers

# # model_name = "gpt2"
# model_name = "microsoft/phi-2"
# model = transformers.AutoModelForCausalLM.from_pretrained(model_name, temperature=0.2, trust_remote_code=True, torch_dtype=torch.bfloat16)
# tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [1]:
import outlines
from concurrent.futures import ThreadPoolExecutor
from human_eval.execution import check_correctness
from human_eval.data import HUMAN_EVAL, stream_jsonl

problems = list(stream_jsonl(HUMAN_EVAL))
executor = ThreadPoolExecutor(max_workers=5)

## TODO: should be returning fraction of test cases passed, instead of 0/1
def execute(problem, completion, timeout):
    future = executor.submit(check_correctness, problem, completion, timeout)
    result = future.result()
    return result


def stats_execute(problem, completion, timeout):
    pre_base_str, tests = problem['test'].split('def check(candidate):\n')
    base_str = "def check(candidate):\n"
    split_tests = [pre_base_str + base_str + i for i in tests.split('\n') if i != '']
    
    _problem = problem.copy()
    results = []
    for i in split_tests:
        _problem['test'] = i
        future = executor.submit(check_correctness, _problem, completion, timeout)
        result = future.result()
        results.append(result)
    
    
    return {'task_id': problem['task_id'], 'pass_rate': sum([i['passed'] for i in results])/len(results)}

In [2]:
@outlines.prompt
def few_shots(instructions, examples, question):
    """{{ instructions }}

    {% for example in examples %}
    Question:
    ```
    {{ example.prompt }}
    ```
    Answer:
    ```
    {{ example.canonical_solution }}
    ```
    {% endfor %}

    Question:
    ```
    {{ question }}
    ```
    Answer:
    ```
    """

instructions = "Please answer the following question following the examples. Generate valid python code by indenting 4 spaces always."
examples = problems[:2]

In [3]:
from graphviz import Digraph
from llama_cpp import Llama
from math import exp, log, inf, sqrt

model = Llama(
    model_path="../deepseek-coder-6.7b-instruct.Q5_K_M.gguf",
    n_gpu_layers=-1,
    n_ctx=2048,
    n_batch=256,
    n_threads=8,
    logits_all=True,
)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ../deepseek-coder-6.7b-instruct.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32256,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q6_K     [ 11008,  4096

In [11]:
max_rollouts = 10
top_k = 3
beam_width = 1
# cache of generated programs => rewards
program_dict = {}
# hyperparameters for P-UCB function
c_base = 10
c = 4


class Node:
    def __init__(self, label, logprob, state, parent):
        self.value = 0
        self.label = label  # token label text for visualisation
        self.prob = exp(logprob)  # necessary for P-UCB calculation
        self.state = state  # full generated text
        self._children = []
        self._parent = parent
        self.visits = 0
        self.p_ucb = 0

    def backprop(self, value):
        if value > self.value:  # TODO: confirm whether this is correct
            self.value = value
            if self._parent is not None:
                self._parent.backprop(value)


In [5]:


# Implements P-UCB heuristic as defined in https://arxiv.org/pdf/2303.05510.pdf#subsection.D.1
# P-UCB-SELECT(s, c) = argmax_a P-UCB(s, a)
# -> where P-UCB(s, a) = Q(s, a) + ß(s) * P(a|s) * √log(s.visits) / (1 + s'.visits)
# -> where ß(s) = log((s.visits + c_base + 1) / c_base) + c
# -> c_base & c are hyperparameters, set to values c_base = 10 & c = 4
def p_ucb_select(parent_node, child_nodes):
    s_visits = parent_node.visits
    beta = log((s_visits + c_base + 1) / c_base) + c

    max_p_ucb = -inf
    max_node = None
    for i in range(len(child_nodes)):
        node = child_nodes[i]
        p_ucb = node.value + beta * node.prob * sqrt(log(s_visits)) / (1 + node.visits)
        if p_ucb > max_p_ucb:
            max_node = node
            max_p_ucb = p_ucb
    
    max_node.p_ucb = max_p_ucb
    return max_node


def get_top_k_tokens(curr_node, k):
    output = model(prompt=curr_node.state, max_tokens=1, temperature=0.2, logprobs=k)
    output_probs = output["choices"][0]["logprobs"]["top_logprobs"][0]
    return output_probs.items()


# TODO: this will have to be updated using low-level llama-cpp-python APIs for
# beam width > 1. May consider switching to HF transformers library if overly
# complex, though this precludes the use of any quantised models
def beam_search(curr_node):
    '''
    This return the complete the program, we gotta index out the question prompt
    '''
    output = model(
        prompt=curr_node.state,
        max_tokens=100,
        temperature=0.2,
        top_k=beam_width,
        stop=["```"],  # example stop sequence
    )
    output_text = output["choices"][0]["text"]
    return curr_node.state + output_text


def calculate_reward(problem, res):
    stats = stats_execute(problem, res, 10)
    # return random.random()
    return stats['pass_rate']


In [6]:
problem_idx = 9
problem = problems[problem_idx]
question = problem['prompt']

prompt = few_shots(instructions, examples, question)
print(prompt)

Please answer the following question following the examples. Generate valid python code by indenting 4 spaces always.

Question:
```
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

```
Answer:
```
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False

```
Question:
```
from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separat

In [13]:
root = Node("<PD>", log(1), prompt, None)
for i in range(max_rollouts):
    curr_node = root
    curr_node.visits += 1
    # selection
    while len(curr_node._children) > 0:
        curr_node = p_ucb_select(curr_node, curr_node._children)
        curr_node.visits += 1

    # expansion
    tokens = get_top_k_tokens(curr_node, top_k)
    child_nodes = [
        Node(token, logprob, curr_node.state + token, curr_node)
        for (token, logprob) in tokens
    ]
    curr_node._children = child_nodes

    # evaluation / simulation
    # TODO: fix beam search
    generated_program = beam_search(curr_node)
    completion = generated_program.replace(prompt, '')
    reward = calculate_reward(problem, completion)

    # backprop
    curr_node.backprop(reward)

    # TODO: early termination if fully accurate program has been generated
    if reward == 1:
        pass

    # break


Llama.generate: prefix-match hit

llama_print_timings:        load time =   30828.89 ms
llama_print_timings:      sample time =       0.25 ms /     1 runs   (    0.25 ms per token,  4016.06 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =     505.66 ms /     1 runs   (  505.66 ms per token,     1.98 tokens per second)
llama_print_timings:       total time =     512.83 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =   30828.89 ms
llama_print_timings:      sample time =      11.89 ms /    54 runs   (    0.22 ms per token,  4540.10 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   16588.41 ms /    54 runs   (  307.19 ms per token,     3.26 tokens per second)
llama_print_timings:       total time =   16

In [15]:
print(completion)


    if not numbers:
        return []

    result = [numbers[0]]
    for num in numbers[1:]:
        result.append(max(result[-1], num))

    return result




In [16]:
res = completion
print(stats_execute(problem, res, 10))
print('---')
print(res)
print('---')
print(problem['canonical_solution'])

{'task_id': 'HumanEval/9', 'pass_rate': 1.0}
---

    if not numbers:
        return []

    result = [numbers[0]]
    for num in numbers[1:]:
        result.append(max(result[-1], num))

    return result


---
    running_max = None
    result = []

    for n in numbers:
        if running_max is None:
            running_max = n
        else:
            running_max = max(running_max, n)

        result.append(running_max)

    return result



In [14]:

##### Visualisation #####


def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()

    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._children:
                edges.add((child, v))
                build(child)

    build(root)
    return nodes, edges


def draw_dot(root):
    dot = Digraph(format="svg", graph_attr={"rankdir": "TB"})  # TB = top to bottom
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(
            name=uid,
            label="{ %s | value %.4f | p_ucb %.4f | visits %d | prob %.4f }"
            % (
                # HACK: quick-fix to escape special chars for label processing
                repr(n.label)
                .replace(">", "\>")
                .replace("<", "\<")
                .replace("}", "\}")
                .replace("{", "\{"),
                n.value,
                n.p_ucb,
                n.visits,
                n.prob,
            ),
            shape="record",
        )
    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n2)), str(id(n1)))
    return dot


dot = draw_dot(root)
dot.render(filename="tree", view=True)


'tree.svg'